In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten, Conv1D
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import pandas as pd
from CLayer.CLayer import *

In [11]:
from sklearn.preprocessing import MinMaxScaler
from random import shuffle

def get_filepaths(mainfolder):
    """
    Searches a folder for all unique files and compile a dictionary of their paths.
    Parameters
    --------------
    mainfolder: the filepath for the folder containing the data
    Returns
    --------------
    training_filepaths: file paths to be used for training
    testing_filepaths:  file paths to be used for testing
    """
    training_filepaths = {}
    testing_filepaths  = {}
    folders = os.listdir(mainfolder)
    for folder in folders:
        fpath = mainfolder + "/" + folder
        if os.path.isdir(fpath) and "MODEL" in folder:
            filenames = os.listdir(fpath)
            for filename in filenames[:int(round(0.8*len(filenames)))]:
                fullpath = fpath + "/" + filename
                training_filepaths[fullpath] = folder
            for filename1 in filenames[int(round(0.8*len(filenames))):]:
                fullpath1 = fpath + "/" + filename1
                testing_filepaths[fullpath1] = folder
    return training_filepaths, testing_filepaths

def get_labels(mainfolder):
    """ Creates a dictionary of labels for each unique type of motion """
    labels = {}
    label = 0
    for folder in os.listdir(mainfolder):
        fpath = mainfolder + "/" + folder
        if os.path.isdir(fpath) and "MODEL" in folder:
            labels[folder] = label
            label += 1
    return labels

def get_data(fp, labels, folders, norm, std, center):
    """
    Creates a dataframe for the data in the filepath and creates a one-hot
    encoding of the file's label
    """
    data = pd.read_csv(filepath_or_buffer=fp, sep=' ', names = ["X", "Y", "Z"])
    if norm and not std:
        normed_data = norm_data(data)
    elif std and not norm:
        stdized_data = std_data(data)
    elif center and not norm and not std:
        cent_data = subtract_mean(data)

    one_hot = np.zeros(7)
    file_dir = folders[fp]
    label = labels[file_dir]
    one_hot[label] = 1
    return normed_data, one_hot, label

# Normalizes the data by removing the mean

def subtract_mean(input_data):
    # Subtract the mean along each column
    centered_data = input_data - input_data.mean()
    return centered_data


def norm_data(data):
    """
    Normalizes the data.
    For normalizing each entry, y = (x - min)/(max - min)
    """
    c_data = subtract_mean(data)
    mms = MinMaxScaler()
    mms.fit(c_data)
    n_data = mms.transform(c_data)
    return n_data

def standardize(data):
    c_data = subtract_mean(data)
    std_data = c_data/ pd.std(c_data)
    return std_data

def vectorize(normed):
    """
    Uses a sliding window to create a list of (randomly-ordered) 300-timestep
    sublists for each feature.
    """
    sequences = [normed[i:i+150] for i in range(len(normed)-150)]
    shuffle(sequences)
    sequences = np.array(sequences)
    return sequences

def build_inputs(files_list, accel_labels, file_label_dict, norm_bool, std_bool, center_bool):
    X_seq    = []
    y_seq    = []
    labels = []
    for path in files_list:
        normed_data, target, target_label = get_data(path, accel_labels, file_label_dict, norm_bool, std_bool, center_bool)
        input_list = vectorize(normed_data)
        for inputs in range(len(input_list)):
            X_seq.append(input_list[inputs])
            y_seq.append(list(target))
            labels.append(target_label)
    X_ = np.array(X_seq)
    y_ = np.array(y_seq)
    return X_, y_, labels

mainpath = "../data/HMP_Dataset"


activity_labels                  = get_labels(mainpath)
training_dict, testing_dict      = get_filepaths(mainpath)
training_files                   = list(training_dict.keys())
testing_files                    = list(testing_dict.keys())

    # build training inputs and labels
X_train, y_train, train_labels = build_inputs(
    training_files,
    activity_labels,
    training_dict,
    True, False, False)


X_test, y_test, test_labels = build_inputs(
    testing_files,
    activity_labels,
    testing_dict,
    True, False, False)

shuffle = np.random.permutation(len(X_train))
X_train = X_train[shuffle]
y_train = y_train[shuffle]

In [12]:
tf.random.set_seed(1234)
model = Sequential([
    Input(X_train.shape[1:]),
    CNN1D(10, 4),
    Flatten(),
    Dense(7, activation='softmax')   
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.fit(X_train, y_train, epochs=10, validation_data = (X_test, y_test))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cn_n1d (CNN1D)               (None, 147, 10)           130       
_________________________________________________________________
flatten (Flatten)            (None, 1470)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 10297     
Total params: 10,427
Trainable params: 10,427
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
294/294 [==============================] - 63s 213ms/step - loss: 0.7011 - accuracy: 0.7675 - val_loss: 0.6211 - val_accuracy: 0.8367
Epoch 2/10
294/294 [==============================] - 63s 213ms/step - loss: 0.3308 - accuracy: 0.8903 - val_loss: 0.3862 - val_accuracy: 0.8793
Epoch 3/10
294/294 [==============================] - 61s 209ms/step - loss: 0.2448 - accura

In [ ]:
tf.random.set_seed(1234)
model = Sequential([
    Input(X_train.shape[1:]),
    Conv1D(10,3, activation='relu'),
    Flatten(),
    Dense(7, activation='softmax')   
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.fit(X_train, y_train, epochs=10, validation_data = (X_test, y_test))

In [ ]:
tf.random.set_seed(1234)
model = Sequential([
    Input(X_train.shape[1:]),
    DWA_CNN(10, 3),
    Flatten(),
    Dense(7, activation='softmax')   
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.fit(X_train, y_train, epochs=10, validation_data = (X_test, y_test))